# Test: the impact of SNR
---
What effect does the SNR have upon the recovery of the mass-weighted age of the stellar population? In this notebook, we will run our ppxf pipline on a series of spectra varying in S/N ratio. 

In [1]:
%matplotlib widget

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))
display(HTML("<style>.output_result { max-width:75% !important; }</style>"))

In [3]:
import numpy as np
from numpy.random import RandomState
from time import time 
from tqdm.notebook import tqdm
import multiprocessing
import pandas as pd

from astropy.io import fits

from ppxftests.run_ppxf import run_ppxf
from ppxftests.ssputils import load_ssp_templates
from ppxftests.mockspec import load_sfh, create_mock_spectrum, calculate_mw_age
from ppxftests.ppxf_plot import plot_sfh_mass_weighted

import matplotlib.pyplot as plt
plt.ion()
plt.close("all")

from IPython.core.debugger import Tracer

In [4]:
###########################################################################
# Settings
###########################################################################
isochrones = "Padova"
sigma_star_kms = 250
z = 0.01

In [ ]:
###########################################################################
# Generate the input SFH
###########################################################################
# Load the stellar templates so we can get the age & metallicity dimensions
_, _, metallicities, ages = load_ssp_templates(isochrones)
N_ages = len(ages)
N_metallicities = len(metallicities)

Galaxies with old stellar populations that we can use: 
1, 2, 6, 11, 24,
Notes:
* 2 - good one to check, as it has a single burst of SF at ~50 Myr and not much else.
* 6 - another good one; SF slowly tapers off until ~20 Myr
* 29, 41 - similar to 6
* 24, 31, 51, 55, 58 - quiescent 
* 42, 53 - scattering of SF events in last ~50 Myr - 1 Gyr


In [7]:
i = 0

In [73]:
# Load a realistic SFH
plt.close("all")
sfh_mw_original = load_sfh(i, plotit=True)
i += 1

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [67]:
###########################################################################
# Check the spectrum and SFH
###########################################################################
spec_original, spec_original_err, lambda_vals_A = create_mock_spectrum(
    sfh_mass_weighted=sfh_mw_original,
    isochrones=isochrones, z=z, SNR=100, sigma_star_kms=sigma_star_kms,
    plotit=True)  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## S/N TESTING
---
In each iteration, run ppxf on a spectrum with a fixed input SFH but varying S/N.

In [45]:
# Helper function for multiprocessing
def ppxf_helper(args):
    # Unpack arguments
    seed, spec, spec_err, lambda_vals_A = args
    
    # Add "extra" noise to the spectrum
    rng = RandomState(seed)
    noise = rng.normal(scale=spec_err)
    spec_noise = spec + noise

    # This is to mitigate the "edge effects" of the convolution with the LSF
    spec_noise[0] = -9999
    spec_noise[-1] = -9999

    # Run ppxf
    pp = run_ppxf(spec=spec_noise, spec_err=spec_err, lambda_vals_A=lambda_vals_A,
                  z=z, ngascomponents=1,
                  regularisation_method="none", 
                  isochrones="Padova",
                  fit_gas=False, tie_balmer=True,
                  plotit=False, savefigs=False, interactive_mode=False)
    return pp


In [46]:
SNR_vals = [1, 5, 10, 20, 50, 100, 1000, 10000]
# SNR_vals = [10, 1000]
log_age_mw_regul_list = []
log_age_mw_MC_list = []
log_age_mw_MC_err_list = []

# DataFrame for storing results
df = pd.DataFrame()

# Figure for plotting & comparing spectra
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 4))
ax.set_xlabel("Observed wavelength (Å)")
ax.set_ylabel(r"$F_\lambda(\lambda)\,(erg\,s^{-1}\,Å^{-1})$")

# ppxf settings
niters = 20
nthreads = 20

for ss, SNR in enumerate(SNR_vals):
    ###########################################################################
    # Create spectrum
    ###########################################################################
    spec, spec_err, lambda_vals_A = create_mock_spectrum(
        sfh_mass_weighted=sfh_mw_original,
        isochrones=isochrones, z=z, SNR=SNR, sigma_star_kms=sigma_star_kms,
        plotit=False)

    # Compute mass-weighted mean age of the stellar pop. < 1 Gyr old
    log_age_mw_original, log_age_mw_original_idx = calculate_mw_age(sfh_mw_original, age_thresh=1e9, ages=ages)

    # Add to plot 
    ax.errorbar(x=lambda_vals_A, y=spec, yerr=spec_err, label=f"SNR = {SNR:d}")

    ###########################################################################
    # Run ppxf WITHOUT regularisation, using a MC approach
    ###########################################################################
    # Input arguments
    seeds = list(np.random.randint(low=0, high=100 * niters, size=niters))
    args_list = [[s, spec, spec_err, lambda_vals_A] for s in seeds]

    # Run in parallel
    print(f"Running ppxf on {nthreads} threads...")
    t = time()
    with multiprocessing.Pool(nthreads) as pool:
        pp_list = list(tqdm(pool.imap(ppxf_helper, args_list), total=niters))
    print(f"Elapsed time in ppxf: {time() - t:.2f} s")

    ###########################################################################
    # Compute the mass-weighted age from the MC runs
    ###########################################################################
    log_age_mw_list = []
    for pp in pp_list:
        log_age_mw, _ = calculate_mw_age(pp.weights_mass_weighted, age_thresh=1e9, ages=ages)
        log_age_mw_list.append(log_age_mw)
    log_age_mw_MC = np.nanmean(log_age_mw_list)
    log_age_mw_MC_err = np.nanstd(log_age_mw_list)

    ###########################################################################
    # Add variables to DataFrame
    ###########################################################################
    df = df.append({
        "SNR": SNR,
        "log mass-weighted mean age <1Gyr (input)": log_age_mw_original,
        "log mass-weighted mean age <1Gyr (MC)": log_age_mw_MC,
        "log mass-weighted mean age <1Gyr (MC) error": log_age_mw_MC_err,
    }, ignore_index=True)

ax.legend()
ax.set_yscale("log")
ax.autoscale(axis="x", tight=True, enable=True)
        

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  # This is added back by InteractiveShellApp.init_path()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Running ppxf on 20 threads...


/home/u5708159/python/Modules/ppxftests/mockspec.py:54: RuntimeWarning: invalid value encountered in double_scalars
  log_age_mw = np.nansum(sfh_mw_1D[:age_thresh_idx] * np.log10(ages[:age_thresh_idx])) / np.nansum(sfh_mw_1D[:age_thresh_idx])



Elapsed time in ppxf: 29.62 s
Running ppxf on 20 threads...



Elapsed time in ppxf: 30.78 s


In [47]:
########################################################
# Plot the recovered mass-weighted mean age vs. SNR
########################################################
fig = plt.figure(figsize=(10, 4))
ax = fig.add_axes([0.1, 0.15, 0.7, 0.75])

# Actual value
ax.axhline(df["log mass-weighted mean age <1Gyr (input)"].unique()[0], color="gray", label="Actual value")

# Results from MC runs
ax.errorbar(x=df["SNR"].values, 
            y=df["log mass-weighted mean age <1Gyr (MC)"].values, 
            yerr=df["log mass-weighted mean age <1Gyr (MC) error"].values,
            linestyle="none", marker="D", label="MC estimate", zorder=999)

# Decorations
ax.set_xlabel(r"S/N ratio")
ax.set_ylabel(r"Mass-weighted mean age < 1 Gyr (log yr)")
ax.grid()
ax.legend(bbox_to_anchor=[1.05, 0.5], loc="center left")
ax.set_title(r"Effect of S/N ratio")
ax.set_xscale("log")



/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  after removing the cwd from sys.path.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …